In [ ]:
!pip install dbt-core

In [ ]:
!pip install dbt-bigquery

In [ ]:
!dbt --version

# Imagens + Query SQL

![ex.1](/images/1-orders-by-year.png) 

In [ ]:
{{ config(materialized='table') }}

SELECT
    EXTRACT(YEAR FROM DATE(order_approved_at)) as year_order,
    count(EXTRACT(YEAR FROM DATE(order_approved_at))) AS num_orders 
FROM 
    `dbt_shop.orders`
WHERE EXTRACT(YEAR FROM DATE(order_approved_at)) is not null
GROUP BY 1

![ex.2](/images/2-orders-by-state.png)

In [ ]:
SELECT
    c.customer_state,
    count(EXTRACT(YEAR FROM DATE(o.order_approved_at))) AS num_orders 
FROM 
    `dbt_shop.orders` as o inner join `dbt_shop.customers`as c on o.customer_id = c.customer_id

WHERE 
    EXTRACT(YEAR FROM DATE(o.order_approved_at)) is not null
GROUP BY 1

![ex.3](/images/3-orders-by-state-greater-5k.png)

In [ ]:
SELECT * FROM (SELECT
    c.customer_state,
    IF(
        count(EXTRACT(YEAR FROM DATE(o.order_approved_at)))>{{ var("quantity_orders") }},
        count(EXTRACT(YEAR FROM DATE(o.order_approved_at))),
        NULL
        ) AS num_orders 
FROM 
    `dbt_shop.orders` as o inner join `dbt_shop.customers`as c on o.customer_id = c.customer_id

WHERE 
    EXTRACT(YEAR FROM DATE(o.order_approved_at)) is not null
GROUP BY 1) as TB_AUX
WHERE num_orders IS NOT NULL

![ex.4](/images/4-customers-by-region.png)

In [ ]:
SELECT
    CASE 
        WHEN customer_state IN ('SP','RJ','MG','ES') THEN 'SUDESTE'
        WHEN customer_state IN ('PR','SC','RS') THEN 'SUL'
        WHEN customer_state IN ('MS','GO','MT','DF') THEN 'CENTRO-OESTE'
        WHEN customer_state IN ('BA','PI','MA','CE','RN','PB','PE','AL','SE') THEN 'NORDESTE'
        WHEN customer_state IN ('AC','RO','AM','RR','PA','TO','AP') THEN 'NORTE'
        ELSE 'NULL' END AS REGIAO,
        count(EXTRACT(YEAR FROM DATE(o.order_approved_at))) AS num_orders
FROM      
    `dbt_shop.orders` as o inner join `dbt_shop.customers`as c on o.customer_id = c.customer_id
WHERE
    EXTRACT(YEAR FROM DATE(o.order_approved_at)) is not null
GROUP BY 1

![ex.5](/images/5-items-costs.png)

In [ ]:
{{ config(materialized='table') }}

SELECT 
    DISTINCT product_id as item,
    (price + freight_value) as valor
FROM 
    `dbt_shop.items`

![ex 6](/images/6-orders-shipped-costs.png)

In [ ]:
SELECT 
    DISTINCT i.order_id,
    (i.price + i.freight_value) as valor
FROM
    `dbt_shop.items` i inner join `dbt_shop.orders` o on i.order_id = o.order_id
WHERE o.order_status = 'shipped'

# Comando dbt via terminal

In [ ]:
dbt run --models shop.orders_by_year

dbt run --models shop.orders_by_state

dbt run --models shop.orders_by_state_greater_than_5k --vars "{'quantity_orders':5000}"

dbt run --models shop.customers_by_region

dbt run --models shop.items_costs

dbt run --models shop.orders_shipped_costs